In [29]:
import splitfolders
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import ann

from typing import Any, Tuple
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import os

## Разделение данных

In [30]:
splitfolders.ratio('dataset', 'data_set_for_lerning',ratio=(0.8, 0.1, 0.1), seed=13, group_prefix=None)

Copying files: 2270 files [00:03, 603.86 files/s]


##  Модель сверточной нейронной сети

In [31]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()

        self.conv_1 = nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2)
        self.conv_3 = nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.max_pool = nn.MaxPool2d(2)

        self.fc_1 = nn.Linear(576, 10)
        self.fc_2 = nn.Linear(10, 1)

    def forward(self, x: torch.tensor) -> torch.tensor:
        output = self.relu(self.conv_1(x))
        output = self.max_pool(output)
        output = self.relu(self.conv_2(output))
        output = self.max_pool(output)
        output = self.relu(self.conv_3(output))
        output = self.max_pool(output)

        
        output = torch.nn.Flatten()(output)
        output = self.relu(self.fc_1(output))
        output = torch.nn.Sigmoid()(self.fc_2(output))
        return output

## Предобработка данных

In [32]:
class CustomImageDataset(Dataset):
  def __init__(self, path_to_annotation_file: str, transform: Any = None, target_transform: Any = None) -> None:
      self.path_to_annotation_file = path_to_annotation_file
      self.dataset_info = pd.read_csv(path_to_annotation_file, delimiter=';')
      self.transform = transform
      self.target_transform = target_transform

  def __len__(self) -> int:
      return len(self.dataset_info)

  def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
      path_to_image = self.dataset_info.iloc[index, 0]
      image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
      label = self.dataset_info.iloc[index, 1]

      if self.transform:
          image = self.transform(image)
      if self.target_transform:
          label = self.target_Transform(label)

      return image, label

## Пейплайн

In [39]:
trans = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize((224, 224)), 
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
train_dataset = CustomImageDataset("annotation_train.csv", trans)
test_dataset = CustomImageDataset("annotation_test.csv", trans)
val_dataset = CustomImageDataset("annotation_val.csv", trans)

## Cuda

In [40]:
device = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = CNN().to(device)
torch.cuda.is_available()

False

## Пейплайн

In [41]:
train_dataset = CustomImageDataset('annotation_train.csv', trans)
test_dataset = CustomImageDataset('annotation_test.csv', trans)
val_dataset = CustomImageDataset('annotation_val.csv', trans)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)


## Первый цикл обучения

In [43]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
criterion = nn.BCELoss()
epochs = 9
model.train()
accuracy_values = []
accuracy_val_values = []
loss_values = []
loss_val_values = []
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    epoch_val_accuracy = 0
    epoch_val_loss = 0

    for data, label in train_dataloader:
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label.unsqueeze(dim=1).to(torch.float))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = np.array(([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(
            label[j]) else 0 for j in range(label.shape[0])])).mean()
        epoch_accuracy += acc / len(train_dataloader)
        epoch_loss += loss / len(train_dataloader)

    accuracy_values.append(epoch_accuracy)
    loss_values.append(epoch_loss)
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(
        epoch + 1, epoch_accuracy, epoch_loss))

    model.eval()
    for data, label in val_dataloader:
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss_val = criterion(output, label.unsqueeze(dim=1).to(torch.float))
        acc_val = np.array(([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(
            label[j]) else 0 for j in range(label.shape[0])])).mean()
        epoch_val_accuracy += acc_val / len(val_dataloader)
        epoch_val_loss += loss_val / len(val_dataloader)

    accuracy_val_values.append(epoch_val_accuracy)
    loss_val_values.append(epoch_val_loss)

    print(f'Epoch : {epoch + 1}, val accuracy : {epoch_val_accuracy}, val loss : { epoch_val_loss}')

Epoch : 1, train accuracy : 0.7512285012285009, train loss : 0.5219229459762573


KeyboardInterrupt: 